In [11]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord

import utils
import model_spectra as NN
import fitting

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

# Recover labels of mock spectra w/ ground truth labels

# Mask Spectra of Standard Star

In [13]:
# read in the standard wavelength grid onto which we interpolate spectra.
wavelength = utils.load_wavelength_array()

# read in all individual neural networks we'll need. 
NN_coeffs = utils.read_in_neural_network(name='norm_spectra_approx')

# Generate a spectrum from "standard" labels and NN
feh = 0.33
alpha = 0.23
alphafe = alpha - feh
Teff = 5663.6
logg = 4.3
dv = 0.0
real_labels = np.array([alphafe, alpha, alpha, alpha, alpha, alpha, alpha,
                        feh, Teff, logg, dv])
real_spec = NN.get_spectrum_from_neural_net(labels=real_labels, NN_coeffs=NN_coeffs)
data_spec = real_spec + 0.01 * np.random.randn(len(real_spec))
spec_err = 0.01 * np.ones(len(wavelength))

# Apply Mask
kirby_2008_stellar = utils.get_spectral_mask_dict(name='kirby_2008_stellar')
mask = utils.generate_mask_from_dict(**kirby_2008_stellar)
spec_err[mask] = 1e16

# Fit spectrum
popt, pcov, model_spec \
    = fitting.fit_normalized_spectrum_single_star_model(norm_spec = data_spec,
                                                        spec_err = spec_err,
                                                        NN_coeffs = NN_coeffs,
                                                        p0 = None, num_p0 = 1)

def plot(lambda_bounds):
    # zoom in on a small region of the spectrum so we can see what's going on.
    lambda_min = lambda_bounds[0]
    lambda_max = lambda_bounds[1]
    m = (wavelength < lambda_max) & (wavelength > lambda_min)
    
    fig, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(14, 4))
    
    ax1.plot(wavelength[m], data_spec[m], 'k', lw=0.5, label = r'$\mathrm{Data\ Spectra}$')
    ax1.plot(wavelength[m], model_spec[m], 'r--', lw=0.75, label = r'$\mathrm{Best-Fit\ Model}$')
    ax1.set_xlim(lambda_min, lambda_max)
    ax1.set_ylim(0.70,1.10)
    ax1.set_ylabel(r'$\mathrm{Normalized\ Flux}$')
    ax1.legend(loc = 'best', frameon = False, fontsize = 12)
    
    ax2.plot(wavelength[m], data_spec[m]-model_spec[m], 'k', lw=0.5)
    ax2.hlines(0, lambda_min, lambda_max, linestyles='-.')
    ax2.set_ylim(-0.15,0.15)
    ax2.set_xlabel(r'$\mathrm{Wavelength\ [\AA]}$')
    ax2.set_ylabel(r'$\mathrm{Residuals}$')
    
    fig.subplots_adjust(hspace=0)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
    plt.show()
interact(plot, lambda_bounds=widgets.IntRangeSlider(min=6250,max=9500,step=50,value=[8400,8700]))

print('Best Fit Labels:')
print(popt)

print('True Labels:')
print(real_labels)

interactive(children=(IntRangeSlider(value=(8400, 8700), description='lambda_bounds', max=9500, min=6250, step=50), Output()), _dom_classes=('widget-interact',))

Best Fit Labels:
[-0.10 0.24 0.22 0.23 0.25 0.25 0.20 0.33 5667.28 4.31 -0.01]
True Labels:
[-0.10 0.23 0.23 0.23 0.23 0.23 0.23 0.33 5663.60 4.30 0.00]


In [17]:
# Standard Labels from APOGEE
matches = [8, 26, 40]
feh = [0.33, -1.26, -0.06]
alpha = [0.23, -0.36, 0.06]
Teff = [5663.6, 5650.5, 4295.2]
logg = [4.3, 3.64, 2.30]
dv = [0.0, 0.0, 0.0]
target = ['m15', 'm15', 'm71']

# Select Masking Star
i = 0
j = matches[i]


# Restore Observed spectra
D_PayneDir = utils.D_PayneDir
SpectraDir = D_PayneDir + 'spectra/obs_spectra/'
SpectraFile = target[i]+'_Horne.npz'
temp = np.load(SpectraDir + SpectraFile)
obj = temp['obj']
norm_spectra = temp['norm_spec']
spectral_err = temp['spec_err']
RA_Dec = SkyCoord(temp['RA_Dec'])
temp.close()

# read in the standard wavelength grid onto which we interpolate spectra.
wavelength = utils.load_wavelength_array()

# read in all individual neural networks we'll need. 
NN_coeffs = utils.read_in_neural_network(name='norm_spectra_approx')

# Generate a spectrum from "standard" labels and NN
alphafe = alpha[i] - feh[i]
real_labels = np.array([alphafe, alpha[i], alpha[i], alpha[i], alpha[i], alpha[i], alpha[i],
                        feh[i], Teff[i], logg[i], dv[i]])
model_spec = NN.get_spectrum_from_neural_net(labels=real_labels, NN_coeffs=NN_coeffs)

# Restore Observed Spectra
norm_spec = norm_spectra[j]
spec_err = spectral_err[j]

# Generate Mask
Mask_Cut = 0.05

model_err = norm_spec - model_spec
spec_mask = np.argwhere(np.abs(model_err) > Mask_Cut)
spec_err[spec_mask] = 1e16

chi_spec = (norm_spec - model_spec) / spec_err
chi_mask = (-np.abs(chi_spec)).argsort()[:int(0.05 * len(norm_spec)), np.newaxis]
#spec_err[chi_mask] = 1e16

mask = np.unique(np.concatenate((spec_mask,chi_mask), 0))
np.save(D_PayneDir + '/other_data/mask.' + obj[j], mask)

unmasked_wavelength = wavelength[spec_err < 1e16]
spec_masked_wavelength = wavelength[spec_mask]
chi_masked_wavelength = wavelength[chi_mask]
masked_wavelength = wavelength[mask]

def plot_mask(lambda_bounds):
    # Plot Observed and Model Spectra w/ Mask Overplotted
    lambda_min = lambda_bounds[0]
    lambda_max = lambda_bounds[1]
    m = (wavelength < lambda_max) & (wavelength > lambda_min)
    fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, figsize=(14, 8))
    ax1.plot(wavelength[m], norm_spec[m], 'k', lw=0.5, label = r'$\mathrm{Observed\ Spectra}$')
    ax1.plot(wavelength[m], model_spec[m], 'r--', lw=0.5, label = r'$\mathrm{Model\ Spectra}$')
    ax1.set_xlim(lambda_min, lambda_max)
    ax1.set_ylim(0.70,1.10)
    ax1.set_ylabel(r'$\mathrm{Normalized\ Flux}$')
    ax1.legend(loc = 'best', frameon = True, fontsize = 12)
    
    ax2.plot(wavelength[m], norm_spec[m]-model_spec[m], 'k', lw=0.5)
    ax2.vlines(spec_masked_wavelength, -0.15, 0.15, color='r', alpha=0.5)
    ax2.hlines(0, lambda_min, lambda_max, linestyles='-')
    ax2.hlines(Mask_Cut, lambda_min, lambda_max, linestyles=':', color='b')
    ax2.hlines(-Mask_Cut, lambda_min, lambda_max, linestyles=':', color='b')
    ax2.set_ylim(-0.15,0.15)
    ax2.set_xlabel(r'$\mathrm{Wavelength\ [\AA]}$')
    ax2.set_ylabel(r'$\mathrm{Residuals}$')
    
    chi_spec = (norm_spec[m]-model_spec[m])**2/spec_err[m]
    ax3.plot(wavelength[m], chi_spec, 'k', lw=0.5)
    ax3.vlines(chi_masked_wavelength, np.min(chi_spec)-1, np.max(chi_spec)+1, color='r', alpha=0.5)
    ax3.set_ylim(np.min(chi_spec), np.max(chi_spec))
    ax3.set_xlabel(r'$\mathrm{Wavelength\ [\AA]}$')
    ax3.set_ylabel(r'$\chi^2$')
    
    plt.suptitle('Obj: %s\n\
                 RA: %.5f Dec: %.5f' % (obj[j], RA_Dec[j].ra.deg, RA_Dec[j].dec.deg))
    fig.subplots_adjust(hspace=0)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
    plt.show()
interact(plot_mask, lambda_bounds=widgets.IntRangeSlider(min=6250,max=9500,step=50,value=[8400,8700]))

print(real_labels)
print('Unmasked pixels = %i' % len(unmasked_wavelength))
print('(Includes all masks)')
print('Masked pixels = %i' % len(masked_wavelength))
print('(Does not include Telluric mask and masking of blue CCD)')

interactive(children=(IntRangeSlider(value=(8400, 8700), description='lambda_bounds', max=9500, min=6250, step=50), Output()), _dom_classes=('widget-interact',))

[-0.10 0.23 0.23 0.23 0.23 0.23 0.23 0.33 5663.60 4.30 0.00]
Unmasked pixels = 5887
(Includes all masks)
Masked pixels = 5501
(Does not include Telluric mask and masking of blue CCD)


In [20]:
# Standard Labels
matches = [8, 26, 7, 40, 11]
feh = [0.33, -1.26, -0.06, -1.67, -1.6]
alpha = [0.23, -0.36, 0.06, 0.18, 0.47]
Teff = [5663.6, 5650.5, 4295.2, 5247., 4285.]
logg = [4.3, 3.64, 2.30, 3.14, 0.83]
dv = [0.0, 0.0, 0.0, 0.0, 0.0]
target = ['m15', 'm15', 'm71','m13', 'ngc7006']
standard = ['APOGEE', 'APOGEE', 'APOGEE', 'Kirby+ 2008', 'Kirby+ 2008']

# Select Masking Star
for i in range(len(matches)):
    j = matches[i]
    
    
    # Restore Observed spectra
    D_PayneDir = utils.D_PayneDir
    SpectraDir = D_PayneDir + 'spectra/obs_spectra/'
    SpectraFile = target[i]+'_Horne.npz'
    temp = np.load(SpectraDir + SpectraFile)
    obj = temp['obj']
    norm_spectra = temp['norm_spec']
    spectral_err = temp['spec_err']
    RA_Dec = SkyCoord(temp['RA_Dec'])
    temp.close()
    
    # read in the standard wavelength grid onto which we interpolate spectra.
    wavelength = utils.load_wavelength_array()
    
    # read in all individual neural networks we'll need. 
    NN_coeffs = utils.read_in_neural_network(name='norm_spectra_approx')
    
    # Generate a spectrum from "standard" labels and NN
    if standard[i] == 'APOGEE':
        alphafe = alpha[i] - feh[i]
        real_labels = np.array([alphafe, alpha[i], alpha[i], alpha[i], alpha[i], alpha[i], alpha[i],
                                feh[i], Teff[i], logg[i], dv[i]])
    elif standard[i] == 'Kirby+ 2008':
        alphafe = alpha[i]
        alpha[i] = alphafe + feh[i]
        real_labels = np.array([alphafe, alpha[i], alpha[i], alpha[i], alpha[i], alpha[i], alpha[i],
                                feh[i], Teff[i], logg[i], dv[i]])
    model_spec = NN.get_spectrum_from_neural_net(labels=real_labels, NN_coeffs=NN_coeffs)
    
    # Restore Observed Spectra
    norm_spec = norm_spectra[j]
    spec_err = spectral_err[j]
    
    # Generate Mask
    Mask_Cut = 0.05
    
    model_err = norm_spec - model_spec
    spec_mask = np.argwhere(np.abs(model_err) > Mask_Cut)
    spec_err[spec_mask] = 1e16
    
    chi_spec = (norm_spec - model_spec) / spec_err
    chi_mask = (-np.abs(chi_spec)).argsort()[:int(0.05 * len(norm_spec)), np.newaxis]
    #spec_err[chi_mask] = 1e16
    
    mask = np.unique(np.concatenate((spec_mask,chi_mask), 0))
    np.save(D_PayneDir + '/other_data/mask.' + obj[j], mask)
    
    unmasked_wavelength = wavelength[spec_err < 1e16]
    spec_masked_wavelength = wavelength[spec_mask]
    chi_masked_wavelength = wavelength[chi_mask]
    masked_wavelength = wavelength[mask]
    
    def plot_mask(lambda_bounds):
        # Plot Observed and Model Spectra w/ Mask Overplotted
        lambda_min = lambda_bounds[0]
        lambda_max = lambda_bounds[1]
        m = (wavelength < lambda_max) & (wavelength > lambda_min)
        fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, figsize=(14, 8))
        ax1.plot(wavelength[m], norm_spec[m], 'k', lw=0.5, label = r'$\mathrm{Observed\ Spectra}$')
        ax1.plot(wavelength[m], model_spec[m], 'r--', lw=0.5, label = r'$\mathrm{Model\ Spectra}$')
        ax1.set_xlim(lambda_min, lambda_max)
        ax1.set_ylim(0.70,1.10)
        ax1.set_ylabel(r'$\mathrm{Normalized\ Flux}$')
        ax1.legend(loc = 'best', frameon = True, fontsize = 12)
        
        ax2.plot(wavelength[m], norm_spec[m]-model_spec[m], 'k', lw=0.5)
        ax2.vlines(spec_masked_wavelength, -0.15, 0.15, color='r', alpha=0.5)
        ax2.hlines(0, lambda_min, lambda_max, linestyles='-')
        ax2.hlines(Mask_Cut, lambda_min, lambda_max, linestyles=':', color='b')
        ax2.hlines(-Mask_Cut, lambda_min, lambda_max, linestyles=':', color='b')
        ax2.set_ylim(-0.15,0.15)
        ax2.set_xlabel(r'$\mathrm{Wavelength\ [\AA]}$')
        ax2.set_ylabel(r'$\mathrm{Residuals}$')
        
        chi_spec = (norm_spec[m]-model_spec[m])**2/spec_err[m]
        ax3.plot(wavelength[m], chi_spec, 'k', lw=0.5)
        ax3.vlines(chi_masked_wavelength, np.min(chi_spec)-1, np.max(chi_spec)+1, color='r', alpha=0.5)
        ax3.set_ylim(np.min(chi_spec), np.max(chi_spec))
        ax3.set_xlabel(r'$\mathrm{Wavelength\ [\AA]}$')
        ax3.set_ylabel(r'$\chi^2$')
        
        plt.suptitle('Obj: %s\n\
                     RA: %.5f Dec: %.5f' % (obj[j], RA_Dec[j].ra.deg, RA_Dec[j].dec.deg))
        fig.subplots_adjust(hspace=0)
        plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
        plt.show()
    interact(plot_mask, lambda_bounds=widgets.IntRangeSlider(min=6250,max=9500,step=50,value=[8400,8700]))
    
    print(real_labels)
    print('Unmasked pixels = %i' % len(unmasked_wavelength))
    print('(Includes all masks)')
    print('Masked pixels = %i' % len(masked_wavelength))
    print('(Does not include Telluric mask and masking of blue CCD)')

interactive(children=(IntRangeSlider(value=(8400, 8700), description='lambda_bounds', max=9500, min=6250, step=50), Output()), _dom_classes=('widget-interact',))

[-0.10 0.23 0.23 0.23 0.23 0.23 0.23 0.33 5663.60 4.30 0.00]
Unmasked pixels = 5887
(Includes all masks)
Masked pixels = 5501
(Does not include Telluric mask and masking of blue CCD)


interactive(children=(IntRangeSlider(value=(8400, 8700), description='lambda_bounds', max=9500, min=6250, step=50), Output()), _dom_classes=('widget-interact',))

[0.90 -0.36 -0.36 -0.36 -0.36 -0.36 -0.36 -1.26 5650.50 3.64 0.00]
Unmasked pixels = 4408
(Includes all masks)
Masked pixels = 8247
(Does not include Telluric mask and masking of blue CCD)


interactive(children=(IntRangeSlider(value=(8400, 8700), description='lambda_bounds', max=9500, min=6250, step=50), Output()), _dom_classes=('widget-interact',))

[0.12 0.06 0.06 0.06 0.06 0.06 0.06 -0.06 4295.20 2.30 0.00]
Unmasked pixels = 5296
(Includes all masks)
Masked pixels = 6203
(Does not include Telluric mask and masking of blue CCD)


interactive(children=(IntRangeSlider(value=(8400, 8700), description='lambda_bounds', max=9500, min=6250, step=50), Output()), _dom_classes=('widget-interact',))

[0.18 -1.49 -1.49 -1.49 -1.49 -1.49 -1.49 -1.67 5247.00 3.14 0.00]
Unmasked pixels = 6010
(Includes all masks)
Masked pixels = 5157
(Does not include Telluric mask and masking of blue CCD)


interactive(children=(IntRangeSlider(value=(8400, 8700), description='lambda_bounds', max=9500, min=6250, step=50), Output()), _dom_classes=('widget-interact',))

[0.47 -1.13 -1.13 -1.13 -1.13 -1.13 -1.13 -1.60 4285.00 0.83 0.00]
Unmasked pixels = 5740
(Includes all masks)
Masked pixels = 7023
(Does not include Telluric mask and masking of blue CCD)
